In [ ]:
!pip install pdfplumber langchain langchain_community langchain_core langchain_experimental faiss-cpu

In [3]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import RetrievalQA

In [9]:
loader = PDFPlumberLoader("/content/Neural_Network_removed.pdf")
docs = loader.load()

In [10]:
docs

[Document(metadata={'source': '/content/Neural_Network_removed.pdf', 'file_path': '/content/Neural_Network_removed.pdf', 'page': 0, 'total_pages': 1, 'Producer': 'iLovePDF', 'ModDate': 'D:20250722173706Z'}, page_content='Speech and Language Processing. Daniel Jurafsky & James H. Martin. Copyright © 2024. All\nrights reserved. Draft of January 12, 2025.\nCHAPTER\n7 Neural Networks\n“[M]achinesofthischaractercanbehaveinaverycomplicatedmannerwhen\nthenumberofunitsislarge.”\nAlanTuring(1948)“IntelligentMachines”,page6\nNeural networks are a fundamental computational tool for language process-\ning, and a very old one. They are called neural because their origins lie in the\nMcCulloch-Pitts neuron (McCulloch and Pitts, 1943), a simplified model of the\nbiologicalneuronasakindofcomputingelementthatcouldbedescribedinterms\nofpropositionallogic. Butthemodernuseinlanguageprocessingnolongerdraws\nontheseearlybiologicalinspirations.\nInstead, a modern neural network is a network of small computin

In [11]:
text_splitter = SemanticChunker(HuggingFaceBgeEmbeddings())
documents = text_splitter.split_documents(docs)

/tmp/ipython-input-11-3180736174.py:1: LangChainDeprecationWarning: Default values for HuggingFaceBgeEmbeddings.model_name were deprecated in LangChain 0.2.5 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceBgeEmbeddings constructor instead.
  text_splitter = SemanticChunker(HuggingFaceBgeEmbeddings())


In [12]:
embedder = HuggingFaceBgeEmbeddings()

/tmp/ipython-input-12-2364638201.py:1: LangChainDeprecationWarning: Default values for HuggingFaceBgeEmbeddings.model_name were deprecated in LangChain 0.2.5 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceBgeEmbeddings constructor instead.
  embedder = HuggingFaceBgeEmbeddings()


In [13]:
vector = FAISS.from_documents(documents,embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [14]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 4.3 MB/s eta 0:00:00


In [15]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    groq_api_key = "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",
    temperature=0,
    model_name ="deepseek-r1-distill-llama-70b"

)

In [16]:
llm.invoke("what is java")

AIMessage(content='<think>\n\n</think>\n\nJava is a high-level, object-oriented programming language developed by Sun Microsystems (now owned by Oracle Corporation). It was created by James Gosling and first released in 1995. Java is known for its platform independence, meaning that programs written in Java can run on any device that has a Java Virtual Machine (JVM) installed, following the "write once, run anywhere" principle.\n\n### Key Features of Java:\n1. **Object-Oriented**: Java follows the principles of object-oriented programming (OOP), such as encapsulation, inheritance, and polymorphism.\n2. **Platform Independent**: Java code is compiled into an intermediate format called bytecode, which runs on the JVM. This allows Java programs to be executed on any platform that has a JVM.\n3. **Simple and Familiar Syntax**: Java\'s syntax is based on C++ but is simpler and easier to learn, with fewer low-level complexities.\n4. **Robust Security**: Java has built-in security features, m

In [17]:
prompt =""""
1.Use the following piece of context to answer the question at the end.
2.If you don't know the answer, just say that you don't know,"I don't know" but donot makeup an answer on your own.\n.
3.Keep the answer crisp and limited to 3, 4 sentenses

Context: {context}
Question: {question}
"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt)

In [18]:
llm_chain = LLMChain(
    llm=llm,
    prompt=QA_CHAIN_PROMPT,
    callbacks= None,
    verbose = True
    )

/tmp/ipython-input-18-754454184.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(


In [19]:
document_prompt = PromptTemplate(
    input_variables=["page_content","soource"],
    template="Context :\ncontent:{page_content}\nsource;{source}",
)


In [21]:
combine_documents_chain = StuffDocumentsChain(
    llm_chain=llm_chain,

    document_variable_name="context",
    document_prompt=document_prompt,
    callbacks=None,

)


In [22]:
qa = RetrievalQA(
    combine_documents_chain=combine_documents_chain,
    verbose = True,
    retriever=retriever,
    return_source_documents = True)

/tmp/ipython-input-22-1959012833.py:1: LangChainDeprecationWarning: This class is deprecated. Use the `create_retrieval_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/retrieval_qa/
  qa = RetrievalQA(


In [26]:
user_input = "what is deep learning ?"
response=qa({"query": user_input})["result"]



> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:
"
1.Use the following piece of context to answer the question at the end. 
2.If you don't know the answer, just say that you don't know,"I don't know" but donot makeup an answer on your own.
.
3.Keep the answer crisp and limited to 3, 4 sentenses

Context: Context :
content:Draft of January 12, 2025. CHAPTER
7 Neural Networks
“[M]achinesofthischaractercanbehaveinaverycomplicatedmannerwhen
thenumberofunitsislarge.”
AlanTuring(1948)“IntelligentMachines”,page6
Neural networks are a fundamental computational tool for language process-
ing, and a very old one. They are called neural because their origins lie in the
McCulloch-Pitts neuron (McCulloch and Pitts, 1943), a simplified model of the
biologicalneuronasakindofcomputingelementthatcouldbedescribedinterms
ofpropositionallogic. Butthemodernuseinlanguageprocessingnolongerdraws
ontheseearlybiologicalinspirations. Instead, a modern neural netwo

In [28]:
response

'<think>\nOkay, so I need to figure out what deep learning is based on the provided context. Let me start by reading through the context carefully. \n\nThe context is from a chapter about Neural Networks in a book titled "Speech and Language Processing" by Daniel Jurafsky and James H. Martin. It mentions that neural networks are a fundamental tool for language processing and have a long history, dating back to the McCulloch-Pitts neuron model. However, modern neural networks don\'t rely much on those early biological inspirations anymore.\n\nThe chapter introduces feedforward networks, which are called that because computation moves iteratively from one layer to the next. It also states that modern neural networks are often called deep learning because they have many layers, making them "deep." \n\nComparing neural networks to logistic regression, the text explains that while both share similar mathematics, neural networks are more powerful. A minimal neural network with a single hidde